<a href="https://colab.research.google.com/github/PaNiKKY/Covid_protenase/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [13]:
dataset = pd.read_csv("/content/merged_data_regression.csv", index_col="InChIKey")
print(len(dataset))
dataset.head(2)

699


,pIC50,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
InChIKey,,,,,,,,,,,,,,,,,,,,,
VDCCGGIBUYGMIS-MRXNPFEDSA-N,6.004365,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BESULOSOTMRSOO-UHFFFAOYSA-N,6.004365,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def Remove_useless_descriptor(data, threshold):
    temp = data.copy()
    data2 = (data.drop("pIC50", axis=1))

    des1 = len(data2.columns)

    h = data2.columns.tolist()
    df = data2.values.astype(np.float)
    df = np.array(df)

    STDEV = np.std(df, axis=0)
    idx = [idx for idx, val in enumerate(STDEV) if val > threshold]
    df2 = df[:, idx]
    hx = np.array(h)[idx]

    df = pd.DataFrame(df2, columns=[hx])

    des2 = len(df.columns)

    df['pIC50'] = temp['pIC50'].values

    print ('from Remove useless descriptor...')
    print ("The initial set of %d descriptors has been reduced to %d descriptors." % (des1, des2))

    return df, des1, des2

In [15]:
def correlation(data, threshold):
    data.columns = data.columns.get_level_values(0)  # Flatten the DF to 2 dimensions
    temp = data.copy()
    des3 = len((data.drop('pIC50', axis=1).columns))
    corr = stats.pearsonr
    col_corr = set()  # Set of all the names of deleted columns
    corr_matrix = data.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] >= threshold:
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
                if colname in data.columns:
                    del data[colname]  # deleting the column from the dataset
    des4 = len((data.drop('pIC50', axis=1).columns))
    data['pIC50'] = temp['pIC50'].values

    print ('from Remove correlation...')
    print ("The initial set of %d descriptors has been reduced to %d descriptors." % (des3, des4))

    return data, des3, des4

In [16]:
data, des1, des2 = Remove_useless_descriptor(dataset, 0.05)
data, des3, des4 = correlation(data, 0.9)

<ipython-input-14-54947bdaf3cf>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = data2.values.astype(np.float)


from Remove useless descriptor...
The initial set of 881 descriptors has been reduced to 545 descriptors.
from Remove correlation...
The initial set of 545 descriptors has been reduced to 297 descriptors.


In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [18]:
x = dataset.drop("pIC50", axis=1)
y = dataset.pIC50

x.shape

(699, 881)

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [54]:
from lightgbm import LGBMRegressor

LG = LGBMRegressor(max_depth=3)
LG_pre = LG.fit(x_train, y_train)

In [29]:
def cal_score(model, x_train, y_train, x_test, y_test):
  np.random.seed(42)

  y_train_pred = model.predict(x_train)
  y_pred = model.predict(x_test)

  print("Score from train set")

  r2_train = r2_score(y_train, y_train_pred)
  mae_train = mean_absolute_error(y_train, y_train_pred)
  mse_train = mean_squared_error(y_train, y_train_pred)

  print(f"R^2 : {r2_train:.4f}  std : {np.std(r2_train):.4f}\nMAE : {mae_train:.4f} std : {np.std(mae_train):.4f}\nMSE : {mse_train:.4f}  std : {np.std(mse_train):.4f}\n")

  cv_R2 = cross_val_score(model, x_train, y_train, cv=10, scoring= "r2")
  cv_mae = np.abs(cross_val_score(model, x_train, y_train, cv=10, scoring= "neg_mean_absolute_error"))
  cv_mse = np.abs(cross_val_score(model, x_train, y_train, cv=10, scoring= "neg_mean_squared_error"))

  print("Score from CV")
  print(f"R^2 : {cv_R2.mean():.4f}  std : {np.std(cv_R2):.4f}\nMAE : {cv_mae.mean():.4f}  std : {np.std(cv_mae):.4f}\nMSE : {cv_mse.mean():.4f} std : {np.std(cv_mse):.4f}\n")

  print("Score from test set")
  r2_test = r2_score(y_test, y_pred)
  mae_test = mean_absolute_error(y_test, y_pred)
  mse_test = mean_squared_error(y_test, y_pred)

  print(f"R^2 : {r2_test:.4f} std : {np.std(r2_test):.4f}\nMAE : {mae_test:.4f} std : {np.std(mae_test):.4f}\nMSE : {mse_test:.4f}  std : {np.std(mse_test):.4f}")

In [55]:
cal_score(LG_pre, x_train, y_train, x_test, y_test)

Score from train set
R^2 : 0.7884  std : 0.0000
MAE : 0.3612 std : 0.0000
MSE : 0.2318  std : 0.0000

Score from CV
R^2 : 0.6245  std : 0.0903
MAE : 0.4647  std : 0.0452
MSE : 0.3985 std : 0.1021

Score from test set
R^2 : 0.6417 std : 0.0000
MAE : 0.4909 std : 0.0000
MSE : 0.4380  std : 0.0000
